### Add points where there is material property information for later use with sklearn

In [1]:
import numpy as np
import points2Rfile
from points2Rfile import grid
import dask.array as da #so I can easily save to hdf5, do not actually do any parallel computing in this module
import utm #to convert input data to utm
import os
import numpy as np
import json

In [2]:
jsonFile = "renoRemi.json"
inputGridFile = "renoRemi100m.hdf5"
#nevada is in UTM zone 11 N
#define REMI unit
REMI_UNIT=100
UTMZONEN=11
UTMZONELETTER='N'
VPMIN = 1500 #I do this since the reno basin is generally pretty wet therefor vp should never drop below this

In [4]:
#load the json data and open the grid file
with open(jsonFile) as file: 
    REMIData = json.loads(file.read())

### Velocity model
"""
@article{olsen2003estimation,
  title={Estimation of Q for long-period (> 2 sec) waves in the Los Angeles basin},
  author={Olsen, KB and Day, SM and Bradley, CR},
  journal={Bulletin of the Seismological Society of America},
  volume={93},
  number={2},
  pages={627--638},
  year={2003},
  publisher={Seismological Society of America}
}
"""


According to DAY/OLSEN $\frac{Q_s}{V_s} = 0.02$ when $V_s$ < 1-2 Km/s  and  $\frac{Q_s}{V_s} = 0.1$ when $V_s > 1.5 \frac{km}{s}$ which is substantially lower than most labratory soil studies, so you may need to tweak this later.

Also Suggests that $\frac{Q_p}{Q_s} = 1.5$




In [5]:
#get vp,p,qp,qs, using day brocher (this ultimately looked like an better study than day olson)
def getMprops(vs,remi="NO station name defined"):
    vs = vs/(1.0E3) #get q in Km/s
    if(vs>=0.3):
        qs = -16+104.3*vs-25.225*vs**2+8.2184*vs**3 
        qp = 2*qs
    else:
        print("VS was below 300 m/s for station ," + remi +" so using constant value of Qs=13, as suggested by Brocher paper")
        qs=13
        qp=2*qs
        
    #now compute vp from vs, note that I think this relation ship may artificially inflate these values, so may 
    #need tweaking
    vp = 1.16*vs+1.36
    if(vp<(VPMIN/1.0E3)):
        vp=(VPMIN/1.0E3)
        print("setting vp to vpmin val of " + str(VPMIN) + "for station" + remi)
    #now compute density--Note I took this from pySW4 since I couldnt actually find this in the brocher paper
    p = (1.6612 * vp
       - 0.4721 * vp**2
       + 0.0671 * vp**3
       - 0.0043 * vp**4
       + 0.000106 * vp**5)
    return 1.0E3*vp,1.0E3*vs,1.0E3*p,qp,qs

In [7]:
#load the grid file
gridFile = grid.grid(fname=inputGridFile)
DX,DY = gridFile.mdata["DX"],gridFile.mdata["DY"]
NX,NY = gridFile.mdata["NX"],gridFile.mdata["NY"]
#load the DEM from the rFile so that I can convert depth to elevation for all of the points
topo = gridFile.topo.compute()

In [8]:
def getNearestTopoPoint(x,y,topo=topo,dx=DX,dy=DY):
    return topo[int(x/dx),int(y/dy)]


In [9]:
#process each station
station={}
xDomain = gridFile.mdata["NX"]*gridFile.mdata["DX"]
yDomain = gridFile.mdata["NY"]*gridFile.mdata["DY"]
coordinates = utm.from_latlon(latitude=gridFile.mdata["LAT0"],longitude=gridFile.mdata["LON0"],force_zone_letter=UTMZONELETTER,force_zone_number=UTMZONEN)
x,y = 0,0
x0,y0 = coordinates[0] ,coordinates[1]
test = '1003-354'
for remi in REMIData:
    #convert station locations to UTM
    try:
        startPoint = REMIData[remi]['Lat/long'].split('/')
        startPoint[0],startPoint[1] =np.float32(startPoint[0]),np.float32(startPoint[1]) 
        coordinates = utm.from_latlon(latitude=startPoint[0],longitude=startPoint[1],force_zone_letter=UTMZONELETTER,force_zone_number=UTMZONEN)
        startPoint[0],startPoint[1] = coordinates[0],coordinates[1]
        siteName = REMIData[remi]['Site name']
    except KeyError:
        #print the file name 
        print(remi)
        break
    except ValueError:
        print(remi)
        break
    #discard the station if it is outside of the  model domain
    if((startPoint[0]-x0)<0 or (startPoint[1]-y0)<0 or (startPoint[0]-x0)>NX*DX or (startPoint[1]-y0)>NY*DY):
        #dont do anything this point is out of the model domain and I want to drop this station
        print("station " + remi + " outside of model domain, ignoring!")
    else:
        #fix input points to the datum
        x,y = startPoint[0]-x0,startPoint[1]-y0
        station[remi]={}
        station[remi]["name"] = siteName
        #station[remi]["x"] = startPoint[0]
        #station[remi]["y"] = startPoint[1]
        #compute the elevation using the depth and the DEM 
        z = getNearestTopoPoint(x,y)
        station[remi]["data"] = []
        #use the vp,vs relationship to compute vp,p,qp,qs from REMI data
        for i in REMIData[remi]["data"]:
            #the reason that this throws back multiple times is because there are multiple depth points for each station
            vp,vs,p,qp,qs = getMprops(i[1],remi)
            #use dem to get elevation from depth
            z = z-i[0]
            #save this data points 
            #format: x,y,z,vp,vs,p,qp,qs
            station[remi]["data"].append([x,y,z,vp,vs,p,qp,qs])
    pass

VS was below 300 m/s for station ,SF02 so using constant value of Qs=13, as suggested by Brocher paper
VS was below 300 m/s for station ,SF02 so using constant value of Qs=13, as suggested by Brocher paper
VS was below 300 m/s for station ,SPHI-RS so using constant value of Qs=13, as suggested by Brocher paper
setting vp to vpmin val of 1500for stationSPHI-RS
VS was below 300 m/s for station ,SPHI-RS so using constant value of Qs=13, as suggested by Brocher paper
setting vp to vpmin val of 1500for stationSPHI-RS
VS was below 300 m/s for station ,SPHI-RS so using constant value of Qs=13, as suggested by Brocher paper
VS was below 300 m/s for station ,SPHI-RS so using constant value of Qs=13, as suggested by Brocher paper
station CCAD-PO-RS outside of model domain, ignoring!
station EagleValleyMiddleSchool-s2-remi outside of model domain, ignoring!
station MindenGardnerville-s4-remi outside of model domain, ignoring!
VS was below 300 m/s for station ,WGLF-RS so using constant value of Qs

### Insert all of this data as points into the grid file points section


In [10]:
print("warning! this doesnt discard duplicate points right now!\n")
for points in station:
    data = np.array(station[points]["data"])
    x,y,z,vp,vs,p,qp,qs = data[:,0],data[:,1],data[:,2],data[:,3],data[:,4],data[:,5],data[:,6],data[:,7]
    #add the default remi unit to this data set
    units = np.zeros(x.shape)
    units[:] = REMI_UNIT
    surfaces = points2Rfile.grid.points(gridFile,points,json.dumps(station[points]),"p",x,y,z,units
                                   ,vp,vs,p,qp,qs,overwrite=True)


warning! this doesnt discard duplicate points right now!



In [11]:
#verify that the hdf5 file is closed correctly
gridFile.data.close()